In [1]:
import random, math
import numpy as np
import torch

from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

import sys
from os import listdir, mkdir
from PIL import Image
from collections import Counter

from IPython.display import display, clear_output

In [2]:
IMAGE_PATH = '../data/images'
ORIG_LABEL_PATH = '../data/annotations'
LABEL_PATH = '../data/annotations2'
RANDOM_SEED = 1234

In [3]:
def get_class_map():
    ret = {}

    i = 0
    for fname in listdir(ORIG_LABEL_PATH):
        img_class, _ = fname.split('.')
        ret[img_class] = i
        i += 1

    return ret

In [4]:
def train_valid_test_split(train_fr=.7, valid_fr=.3, test_fr=0):
    """
    Creates subfolders train, test, validation. 
    Splits annotation files into train, validation and test sets.
    """
    random.seed(RANDOM_SEED)
    
    try:
        mkdir(f'{LABEL_PATH}')
        mkdir(f'{LABEL_PATH}/train')
        mkdir(f'{LABEL_PATH}/test')
        mkdir(f'{LABEL_PATH}/validation')
    except:
        pass
      
    # Make the split
    for fname in listdir(ORIG_LABEL_PATH):
        with open(f'{ORIG_LABEL_PATH}/{fname}', 'r') as fh:
            img_ids = fh.read().splitlines()
            random.shuffle(img_ids)
            split1, split2 = math.ceil(test_fr * len(img_ids)), math.ceil(valid_fr * len(img_ids))
            test_ids, valid_ids, train_ids = img_ids[:split1], img_ids[split1:split2], img_ids[split2:]
            with open(f'{LABEL_PATH}/test/{fname}', 'w') as outfile:
                outfile.write('\n'.join(test_ids))
            with open(f'{LABEL_PATH}/validation/{fname}', 'w') as outfile:
                outfile.write('\n'.join(valid_ids))
            with open(f'{LABEL_PATH}/train/{fname}', 'w') as outfile:
                outfile.writelines('\n'.join(train_ids))
            print(f'Wrote {len(train_ids)}, {len(valid_ids)}, {len(test_ids)} lines (of {len(img_ids)}) in {fname} (train, validation, test)')

# Need to run only once. 
train_valid_test_split()

Wrote 367, 158, 0 lines (of 525) in tree.txt (train, validation, test)
Wrote 2258, 969, 0 lines (of 3227) in female.txt (train, validation, test)
Wrote 532, 229, 0 lines (of 761) in flower.txt (train, validation, test)
Wrote 313, 135, 0 lines (of 448) in dog.txt (train, validation, test)
Wrote 121, 52, 0 lines (of 173) in sea.txt (train, validation, test)
Wrote 4482, 1921, 0 lines (of 6403) in people.txt (train, validation, test)
Wrote 2184, 937, 0 lines (of 3121) in portrait.txt (train, validation, test)
Wrote 223, 96, 0 lines (of 319) in car.txt (train, validation, test)
Wrote 766, 329, 0 lines (of 1095) in clouds.txt (train, validation, test)
Wrote 2085, 894, 0 lines (of 2979) in male.txt (train, validation, test)
Wrote 84, 36, 0 lines (of 120) in river.txt (train, validation, test)
Wrote 252, 108, 0 lines (of 360) in bird.txt (train, validation, test)
Wrote 418, 180, 0 lines (of 598) in night.txt (train, validation, test)
Wrote 66, 29, 0 lines (of 95) in baby.txt (train, validation

In [37]:
def get_dataloader(set_path, bs=64, max_items=256):
    data = []
    X = []
    y = []

    # mapping from class names to integers
    class_map = get_class_map()

    # loop through all the annotations
    for fname in listdir(set_path):

        img_class, _ = fname.split('.')
        clear_output(wait=True)
        display(f'Reading set: {set_path}, class: {img_class}')
        #print(f'Reading class: {img_class}')

        # open the annotation
        with open(f'{set_path}/{fname}', 'r') as fh:

            # get image ids from annotation file
            img_ids = fh.read().splitlines()

            # gather the images with labels
            i = 0
            for img_id in img_ids:
                img_path = f'{IMAGE_PATH}/im{img_id}.jpg'
                img = Image.open(img_path)
                img_data = np.asarray(img)

                # skip black-and-white images
                if not len(img_data.shape) == 3:
                    continue

                img_data = img_data.flatten().astype(np.float32)

                data.append([img_data, class_map[img_class]])

                if i > max_items: break
                i += 1
                
    return DataLoader(data, batch_size=bs, shuffle=True)

def get_dataloaders(bs=64, max_items=256):
   
    train_loader = get_dataloader(f'{LABEL_PATH}/train', bs, max_items)
    validation_loader = get_dataloader(f'{LABEL_PATH}/validation', bs, max_items)
    #test_loader = get_dataloader(f'{LABEL_PATH}/test', bs, max_items)
    
    print(f'Read {len(train_loader.dataset)}, {len(validation_loader.dataset)} items in train, validation sets')
    
    return train_loader, validation_loader#, test_loader

In [6]:
class TwoLayerModel(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_classes):
        super().__init__()

        self.input_layer = nn.Linear(n_input, n_hidden1)
        self.hidden1 = nn.Linear(n_hidden1, n_hidden2)
        self.hidden2 = nn.Linear(n_hidden2, n_classes)
        self.relu = nn.ReLU()
        self.bn0 = nn.BatchNorm1d(n_input)
        self.bn1 = nn.BatchNorm1d(n_hidden1)
        self.bn2 = nn.BatchNorm1d(n_hidden2)

    def forward(self, x):
        x = self.bn0(x)
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.hidden2(x)

        return x

In [7]:
class OneLayerModel(nn.Module):
    def __init__(self, n_input, n_hidden, n_classes):
        super().__init__()

        self.input_layer = nn.Linear(n_input, n_hidden)
        self.hidden = nn.Linear(n_hidden, n_classes)
        self.relu = nn.ReLU()
        self.bn0 = nn.BatchNorm1d(n_input)
        self.bn1 = nn.BatchNorm1d(n_hidden)

    def forward(self, x):
        x = self.bn0(x)
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.hidden(x)

        return x

In [8]:
def train(dataloader, model, optimizer, criterion, device, n_epochs=50, losses=[]):

    log = []
    model.train()

    for epoch in range(n_epochs):
        
        for i, batch in enumerate(dataloader):
            X, y = batch
            
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            
            losses.append(loss)
            
            clear_output(wait=True)
            display(f'Epoch: {epoch+1}, iteration: {i+1}, loss: {loss}')

        log.append(f'Epoch: {epoch+1}, loss: {loss}')
    
    print('\n'.join(log))
    print('\nDone training')

In [9]:
use_cuda = False

device = torch.device('cuda') if use_cuda else torch.device('cpu')

lr = 0.01
n_epochs = 2 #10
bs = 64 #256
max_items = 256 #sys.maxsize

n_classes = len(get_class_map().keys())

#model = TwoLayerModel(128*128*3, 128, 64, n_classes).to(device)
model = OneLayerModel(128*128*3, 128, n_classes).to(device)   # (self, D_in, H, D_out):

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [38]:
train_dataloader, validation_dataloader = get_dataloaders(bs, max_items)

'Reading set: ../data/annotations2/validation, class: baby'

Read 3034, 2262 items in train, validation sets


In [39]:
train(train_dataloader, model, optimizer, criterion, device, n_epochs)

'Epoch: 1, iteration: 47, loss: 2.074885368347168'

Epoch: 0, loss: 1.9990369081497192
Epoch: 1, loss: 2.074885368347168

Done training
